In [1]:
import re

import pandas as pd
import datetime

from pymatgen.core import Structure, Lattice, Composition
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.ext.matproj import MPRester
from m3gnet.models import Relaxer

In [2]:
data = pd.read_html("http://en.wikipedia.org/wiki/Lattice_constant")[0]
data = data[~data["Crystal structure"].isin(["Hexagonal", "Wurtzite", "Wurtzite (HCP)", 
                                             "Orthorombic", "Tetragonal perovskite", "Orthorhombic perovskite"])]
data.rename(columns = {'Lattice constant (Å)':'a'}, inplace = True)
data.drop(columns=["Ref."], inplace=True)
data["a"] = data["a"].map(float)
data = data[["Material", "Crystal structure", "a"]]
data = data[data["Material"] != "NC0.99"]

additional_fcc = """10 Ne 4.43 54 Xe 6.20
13 Al 4.05 58 Ce 5.16
18 Ar 5.26 70 Yb 5.49
20 Ca 5.58 77 Ir 3.84
28 Ni 3.52 78 Pt 3.92
29 Cu 3.61 79 Au 4.08
36 Kr 5.72 82 Pb 4.95
38 Sr 6.08 47 Ag 4.09
45 Rh 3.80 89 Ac 5.31
46 Pd 3.89 90 Th 5.08"""

additional_bcc = """3 Li 3.49 42 Mo 3.15
11 Na 4.23 55 Cs 6.05
19 K 5.23 56 Ba 5.02
23 V 3.02 63 Eu 4.61
24 Cr 2.88 73 Ta 3.31
26 Fe 2.87 74 W 3.16
37 Rb 5.59 41 Nb 3.30"""

def add_new(str_, structure_type, df):
    toks = str_.split()
    new_crystals = []
    for i in range(int(len(toks) / 3)):
        el = toks[3*i+1].strip()
        if el not in df["Material"].values:
            new_crystals.append([toks[3*i+1], structure_type, float(toks[3*i+2])])
    df2 = pd.DataFrame(new_crystals, columns=data.columns)
    return pd.concat([df, df2])

data = add_new(additional_fcc, "FCC", data)
data = add_new(additional_bcc, "BCC", data)
data.set_index("Material", inplace=True)
print(data)

             Crystal structure         a
Material                                
C (diamond)      Diamond (FCC)  3.567000
Si               Diamond (FCC)  5.431021
Ge               Diamond (FCC)  5.658000
AlAs         Zinc blende (FCC)  5.660500
AlP          Zinc blende (FCC)  5.451000
...                        ...       ...
K                          BCC  5.230000
Ba                         BCC  5.020000
Eu                         BCC  4.610000
Cr                         BCC  2.880000
Rb                         BCC  5.590000

[92 rows x 2 columns]


In [3]:
predicted = []
mp = []
relaxation_time = []
mpr = MPRester()
relaxer = Relaxer()  # This loads the default pre-trained model

for formula, v in data.iterrows():
    formula = formula.split()[0]
    c = Composition(formula)
    els = sorted(c.elements)
    cs = v["Crystal structure"]
    
    # We initialize all the crystals with an arbitrary lattice constant of 5 angstroms.
    if "Zinc blende" in cs:
        s = Structure.from_spacegroup("F-43m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.25, 0.25, 0.75]])
    elif "Halite" in cs:
        s = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0, 0]])
    elif "Caesium chloride" in cs:
        s = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif "Cubic perovskite" in cs:
        s = Structure(Lattice.cubic(5), [els[0], els[1], els[2], els[2], els[2]], 
                      [[0., 0., 0.], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0., 0.5, 0.5], [0.5, 0, 0.5]])
    elif "Diamond" in cs:
        s = Structure.from_spacegroup("Fd-3m", Lattice.cubic(5), [els[0]], [[0.25, 0.75, 0.25]])
    elif "BCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 2, [[0., 0., 0.], [0.5, 0.5, 0.5]])
    elif "FCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 4, [[0., 0., 0.], [0.5, 0.5, 0], [0., 0.5, 0.5], [0.5, 0, 0.5]])
    else:
        predicted.append(0)
        mp.append(0)
        continue
    
    start = datetime.datetime.now()

    relax_results = relaxer.relax(s)

    final_structure = relax_results['final_structure']

    timetaken = datetime.datetime.now() - start
     
    predicted.append(final_structure.lattice.a)
    relaxation_time.append(timetaken)
    
    try:
        entries = mpr.get_entries({"pretty_formula": s.composition.reduced_formula}, 
                                  inc_structure=True, property_data=["e_above_hull"])
        entries = sorted(entries, key=lambda e: e.data["e_above_hull"])
        for e in entries:
            try:
                sga = SpacegroupAnalyzer(e.structure)
                sga2 = SpacegroupAnalyzer(final_structure)
                if sga.get_space_group_number() == sga2.get_space_group_number():
                    conv = sga.get_conventional_standard_structure()
                    mp.append(conv.lattice.a)
                    break
            except:
                pass
        else:
            raise RuntimeError
    except Exception as ex:
        mp.append(0)
        import traceback
        traceback.print_exc()
data["MP a"] = mp
data["Predicted a"] = predicted

/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall:4", shape=(None,), dtype=int32), values=Tensor("PartitionedCall:3", shape=(None, 3, 3), dtype=float32), dense_shape=Tensor("PartitionedCall:5", shape=(3,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall:1", shape=(544,), dtype=int32), values=Tensor("Neg:0", shape=(544, 3), dtype=float32), dense_shape=Tensor("PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
2022-05-31 18:45:57.359344: W tensorflow/core/platform/profile_utils/cpu_

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:45:57      -36.148697*      38.4316


/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Reshape:0", dtype=float32), dense_shape=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Cast:0", shape=(None,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/m3g_net/graph_network_layer_2/gated_atom_update_2/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/m3g_net/graph_network_layer_2/gated_atom_update_2/GatherV2_grad/Reshape

FIRE:    1 18:45:59      -47.801781*      84.2779
FIRE:    2 18:45:59      -67.004982*      61.9953
FIRE:    3 18:45:59      -72.645134*       5.3897
FIRE:    4 18:45:59      -72.689499*       1.7229
FIRE:    5 18:45:59      -72.692436*       1.1855
FIRE:    6 18:45:59      -72.694916*       0.2681
FIRE:    7 18:45:59      -72.694031*       0.7506
FIRE:    8 18:45:59      -72.694176*       0.6899
FIRE:    9 18:45:59      -72.694435*       0.5736
FIRE:   10 18:45:59      -72.694733*       0.4114
FIRE:   11 18:45:59      -72.694962*       0.2167
FIRE:   12 18:45:59      -72.695045*       0.0054
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:46:01      -39.464790*      44.6512
FIRE:    1 18:46:01      -42.685207*       6.6691
FIRE:    2 18:46:02      -42.835457*       2.9842
FIRE:    3 18:46:02      -42.834694*       3.7586
FIRE:    4 18:46:02      -42.849213*       2.8557
FIRE:    5 18:46:02      -42.865707*       1.4411
FI

FIRE:    3 18:46:13      -27.507965*      12.9482
FIRE:    4 18:46:13      -28.111067*       6.6576
FIRE:    5 18:46:13      -28.207924*       5.8834
FIRE:    6 18:46:13      -28.243465*       4.4674
FIRE:    7 18:46:13      -28.284222*       2.3310
FIRE:    8 18:46:13      -28.302393*       0.1859
FIRE:    9 18:46:13      -28.290806*       1.6932
FIRE:   10 18:46:13      -28.291628*       1.6340
FIRE:   11 18:46:13      -28.293159*       1.5171
FIRE:   12 18:46:13      -28.295195*       1.3460
FIRE:   13 18:46:13      -28.297430*       1.1258
FIRE:   14 18:46:13      -28.299562*       0.8627
FIRE:   15 18:46:13      -28.301264*       0.5643
FIRE:   16 18:46:13      -28.302296*       0.2396
FIRE:   17 18:46:13      -28.302444*       0.1368
FIRE:   18 18:46:13      -28.302448*       0.1353
FIRE:   19 18:46:13      -28.302450*       0.1321
FIRE:   20 18:46:13      -28.302460*       0.1274
FIRE:   21 18:46:14      -28.302462*       0.1212
FIRE:   22 18:46:14      -28.302471*       0.1136


...

/Users/shyue/repos/pymatgen/pymatgen/core/periodic_table.py:210: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:47:33       -5.575591*       6.3206
FIRE:    1 18:47:33       -6.066901*       1.8510
FIRE:    2 18:47:33       -6.006678*       1.8936
FIRE:    3 18:47:33       -6.022381*       1.6590
FIRE:    4 18:47:33       -6.047195*       1.3495
FIRE:    5 18:47:33       -6.074849*       1.0743
FIRE:    6 18:47:33       -6.099977*       0.6871
FIRE:    7 18:47:33       -6.110145*       0.1107
FIRE:    8 18:47:33       -6.110159*       0.1065
FIRE:    9 18:47:33       -6.110185*       0.0982


/Users/shyue/repos/pymatgen/pymatgen/core/periodic_table.py:210: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/Users/shyue/repos/pymatgen/pymatgen/core/periodic_table.py:210: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:47:34       -7.027706*      10.6407
FIRE:    1 18:47:34       -7.959613*       0.5016
FIRE:    2 18:47:34       -7.960681*       0.4075
FIRE:    3 18:47:34       -7.962051*       0.2349
FIRE:    4 18:47:34       -7.962717*       0.0133
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:47:35      -17.889702*      27.4870
FIRE:    1 18:47:35      -27.500534*      59.2247
FIRE:    2 18:47:35      -35.472523*      12.5807
FIRE:    3 18:47:35      -28.847599*      87.2047
FIRE:    4 18:47:35      -35.472523*      12.5807
FIRE:    5 18:47:35      -35.629105*       8.6481
FIRE:    6 18:47:35      -35.755444*       1.3892
FIRE:    7 18:47:35      -35.681175*       7.1230
FIRE:    8 18:47:35      -35.695297*       6.4244
FIRE:    9 18:47:35      -35.718151*       5.1118
FIRE:   10 18:47:35      -35.741192*       3.3330
FIRE

FIRE:    8 18:47:45      -20.468721*       0.2482
FIRE:    9 18:47:45      -20.468939*       0.2422
FIRE:   10 18:47:45      -20.469215*       0.2337
FIRE:   11 18:47:45      -20.469543*       0.2224
FIRE:   12 18:47:45      -20.469910*       0.2077
FIRE:   13 18:47:45      -20.470297*       0.1892
FIRE:   14 18:47:45      -20.470720*       0.1635
FIRE:   15 18:47:45      -20.471146*       0.1280
FIRE:   16 18:47:45      -20.471512*       0.0793
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:47:45      -11.394991*       2.0676
FIRE:    1 18:47:45      -11.487667*       2.7445
FIRE:    2 18:47:45      -11.825115*       4.8693
FIRE:    3 18:47:45      -13.155437*       9.6248
FIRE:    4 18:47:45      -15.444618*       7.9539
FIRE:    5 18:47:45      -16.979153*       7.0614
FIRE:    6 18:47:45      -18.954466*       5.5520
FIRE:    7 18:47:45      -18.426111*      12.0357
FIRE:    8 18:47:46      -19.002239*       7.8494
FI

In [4]:
data["% error vs Expt"] = (data["Predicted a"] - data["a"])/data["a"] * 100
data["% error vs MP"] = (data["Predicted a"] - data["MP a"])/data["MP a"] * 100

In [5]:
data["% error vs Expt"] = data["% error vs Expt"].map('{:,.2f}%'.format)
data["% error vs MP"] = data["% error vs MP"].map('{:,.2f}%'.format)

In [6]:
print(data.sort_index().to_markdown())

| Material    | Crystal structure   |       a |    MP a |   Predicted a | % error vs Expt   | % error vs MP   |
|:------------|:--------------------|--------:|--------:|--------------:|:------------------|:----------------|
| Ac          | FCC                 | 5.31    | 5.66226 |       5.6646  | 6.68%             | 0.04%           |
| Ag          | FCC                 | 4.079   | 4.16055 |       4.16702 | 2.16%             | 0.16%           |
| Al          | FCC                 | 4.046   | 4.03893 |       4.04108 | -0.12%            | 0.05%           |
| AlAs        | Zinc blende (FCC)   | 5.6605  | 5.73376 |       5.73027 | 1.23%             | -0.06%          |
| AlP         | Zinc blende (FCC)   | 5.451   | 5.50711 |       5.50346 | 0.96%             | -0.07%          |
| AlSb        | Zinc blende (FCC)   | 6.1355  | 6.23376 |       6.22817 | 1.51%             | -0.09%          |
| Ar          | FCC                 | 5.26    | 5.64077 |       5.62745 | 6.99%             | -0.24%    